In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import streamlit as st
import geopy.distance
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
import osmnx 
import numpy as np
import geocoder
import datapackage
import geopip
import ast
from haversine import haversine, Unit
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# The Big Query Credentials are in the path .streamlit/secrets.toml 
# First we save this credentials in the variable credentials and then we create the connection with bigquery
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
BigQuery_client = bigquery.Client(credentials=credentials)
geolocator = Nominatim(user_agent="geoapiExercises")
# The ofertas query acces the locations of the accounts and the location of the rep when the visit was registered.
datasets = list(BigQuery_client.list_datasets())  # Make an API request.
project = BigQuery_client.project

/Users/danielguerrero/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
dataset_id = 'star-big-data.star_us_rds'
tables = BigQuery_client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'star-big-data.star_us_rds':
star-big-data.star_us_rds.AUDIT_REPORT_offersConsolidated_CA
star-big-data.star_us_rds.AUDIT_REPORT_offersConsolidated_CA_copy
star-big-data.star_us_rds.AUDIT_REPORT_offersConsolidated_US
star-big-data.star_us_rds.AUDIT_REPORT_offersConsolidated_US_copy
star-big-data.star_us_rds.AUDIT_offersConsolidated_US
star-big-data.star_us_rds.CrownsAlloyContentsTable
star-big-data.star_us_rds.CrownsAlloyDiscountsTable
star-big-data.star_us_rds.CrownsCountry
star-big-data.star_us_rds.JewelryCustomer
star-big-data.star_us_rds.Melt
star-big-data.star_us_rds.NEWMASTERSTAR_USD_ADELDA
star-big-data.star_us_rds.PickupRequest
star-big-data.star_us_rds.REPORT_GM_compare_companyWide_report
star-big-data.star_us_rds.REPORT_GM_compare_report
star-big-data.star_us_rds.REPORT_contribution_plan_monthly
star-big-data.star_us_rds.REPORT_contribution_plan_monthly_bkp_1
star-big-data.star_us_rds.REPORT_contribution_plan_monthly_copy
star-big-data.star_us_rds.REPORT_c

In [6]:
from google.cloud import bigquery
print("version: ", bigquery.__version__)

version:  2.26.0


In [2]:
def distance_rows(start, points):
        return min([haversine(start,stop,unit=Unit.FEET) for stop in points])

def get_country(Latitude,Longitude):
    location = geolocator.reverse(str(Latitude)+","+str(Longitude))
    address = location.raw['address']
    return address['country']

In [5]:
cuentas_q="""
SELECT a.id,a.practice_name,p.longitude as acc_lon,p.latitude as acc_lat,p.state,p.city, p.country
FROM `star-big-data.star_us_rds.accounts` a
LEFT JOIN `star-big-data.star_us_rds.places` p ON a.place_id=p.id
"""
ofertas_q="""
SELECT o.id,CAST(o.offer_date as DATE) as date,CAST(o.offer_date as time) as time,o.account_id,a.practice_name,r.name,o.longitude as visit_lon,o.latitude as visit_lat,p.longitude as acc_lon,p.latitude as acc_lat,p.state,p.city,o.status,o.total_offer
FROM `star-big-data.star_us_rds.offers` o
LEFT JOIN `star-big-data.star_us_rds.accounts` a ON o.account_id=a.id
LEFT JOIN `star-big-data.star_us_rds.places` p ON a.place_id=p.id
LEFT JOIN `star-big-data.star_us_rds.reps` r ON o.rep_id = r.id
WHERE r.name <> 'General Pot' and status <> 'NoVisit' and account_id IS NOT NULL and (p.longitude IS NOT NULL and p.latitude IS NOT NULL) and (o.longitude IS NOT NULL and o.latitude IS NOT NULL) and r.inactive=False
ORDER BY offer_date desc
"""
tracks="""
    SELECT date,name, STRING_AGG(coordinates) as coords
    FROM(
        SELECT CAST(timestamp as DATE) as date, r.name, CONCAT("(",latitude,",",longitude,")") as coordinates
        FROM `star-big-data.star_us_rds.Track` t
        LEFT JOIN `star-big-data.star_us_rds.reps` r ON t.rep_id = r.id 
    )
    GROUP BY date,name
"""

#cuentas=(BigQuery_client.query(cuentas_q).result().to_dataframe(create_bqstorage_client=True,))
#cuentas = cuentas.dropna(subset=['acc_lon','acc_lat'])
#cuentas['country_info']=cuentas.apply(lambda row : geopip.search(lng=row["acc_lon"],lat=row["acc_lat"])['NAME'] if geopip.search(lng=row["acc_lon"],lat=row["acc_lat"]) else '', axis=1)

# coordinates=(BigQuery_client.query(tracks).result().to_dataframe(create_bqstorage_client=True,))
# coordinates = coordinates.dropna()
# coordinates['coords']=coordinates['coords'].apply(lambda x: list(ast.literal_eval(x)))

ofertas = (BigQuery_client.query(ofertas_q).result().to_dataframe(create_bqstorage_client=True,))
ofertas =ofertas.loc[:,~ofertas.columns.str.startswith('_')]
ofertas['acc_coords']=ofertas[['acc_lat','acc_lon']].apply(lambda x: (x[0],x[1]),axis=1)
ofertas['visit_coords']=ofertas[['visit_lat','visit_lon']].apply(lambda x: (x[0],x[1]),axis=1)


# consolidado=ofertas.merge(coordinates,how='left',on=['date','name']).dropna()
# consolidado['acc_route_dist']=(consolidado[['acc_coords','coords']].apply(lambda x : distance_rows(x[0],x[1]),axis=1))

In [6]:
ofertas

,id,date,time,account_id,practice_name,name,visit_lon,visit_lat,acc_lon,acc_lat,state,city,status,total_offer,acc_coords,visit_coords
0,1509839,2022-08-16,22:33:38,445695,Great Smile Family Dentistry,Michael Beller,-111.011,32.344,-111.012,32.343,AZ,Tucson,NoOffer,0.000,"(32.34329, -111.0116)","(32.343841552734375, -111.01149752199659)"
1,1509837,2022-08-16,22:29:39,206777,Curtis L Gottfried,Jordan Hunter,-122.423,40.585,-122.423,40.585,CA,Redding,Accepted,250.000,"(40.58481, -122.423391)","(40.584913794128646, -122.4225805735156)"
2,1509835,2022-08-16,22:23:07,443765,Desert Springs Family Dentistry,Michael Beller,-111.013,32.336,-111.013,32.339,AZ,Tucson,NoOffer,0.000,"(32.33918, -111.01263)","(32.33594754253513, -111.01273337828836)"
3,1509834,2022-08-16,22:19:11,509126,Mansion Flats Dental,Elizabeth Reynoso,-121.488,38.581,-121.488,38.582,CA,Sacramento,NoOffer,0.000,"(38.58174, -121.48779)","(38.58138998161185, -121.48835142316219)"
4,1509833,2022-08-16,22:16:54,504856,Columbia Dental Care,Eran Even-Kesef,-121.929,37.488,-121.929,37.488,CA,Fremont,Accepted,220.000,"(37.488312, -121.928955)","(37.48808652638226, -121.92901055564587)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118923,1292247,2020-06-02,16:56:57,17257,Conner Dental,Demery Moody,-84.592,34.001,-84.588,34.000,GA,Kennesaw,NoOffer,0.000,"(34.00036, -84.588372)","(34.000614965366196, -84.59168384790848)"
118924,1292245,2020-06-02,16:38:15,376805,Omega Family Dentistry,Demery Moody,-84.628,34.022,-84.632,34.025,GA,Kennesaw,NoOffer,0.000,"(34.02477, -84.63216)","(34.022480537099234, -84.62784855648582)"
118925,1292244,2020-06-02,15:58:22,17264,Acworth Family Dentistry,Demery Moody,-84.656,34.032,-84.647,34.035,GA,Kennesaw,NoOffer,0.000,"(34.03516, -84.64722)","(34.03190347834865, -84.6564969980015)"
118926,1292243,2020-06-02,15:45:44,265027,Dental Excellence of Kennesaw,Demery Moody,-84.619,34.021,-84.621,34.021,GA,Kennesaw,NoOffer,0.000,"(34.02055, -84.62054)","(34.021040070833656, -84.61933541119528)"


In [4]:
from sklearn.neighbors import DistanceMetric
from math import radians

dist = DistanceMetric.get_metric('haversine')

df_distancia=cuentas.head().set_index('id')[['acc_lat','acc_lon']]
df_distancia['acc_lat'] = np.radians(df_distancia['acc_lat'])
df_distancia['acc_lon'] = np.radians(df_distancia['acc_lon'])

/Users/danielguerrero/miniforge3/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [12]:
df__=pd.DataFrame(dist.pairwise(df_distancia[['acc_lat','acc_lon']].to_numpy())*3798,index=df_distancia.index,columns=df_distancia.index)

In [15]:
df__.loc[df__[536448]<=100,536448].sort_values()

id
536448    0.000
536216   83.507
520616   84.763
Name: 536448, dtype: float64

In [10]:
cuentas[['id','acc_lat','acc_lon']].head()

,id,acc_lat,acc_lon
2,520616,43.006,-71.466
3,536216,43.191,-71.536
4,536448,42.871,-73.203
5,520046,43.182,-70.877
6,409179,43.198,-70.998


In [12]:
ofertas['buy']=(ofertas['status']=='Accepted').astype(int)
ofertas['visit']=1

ofertas['cum_prevbuys'] = ofertas.groupby(['account_id'])['buy'].apply(lambda x: x.shift().cumsum())
ofertas['cum_prevvisits'] = ofertas.groupby(['account_id'])['visit'].apply(lambda x: x.shift().cumsum())

ofertas['cum_prevvisits'] = ofertas['cum_prevvisits'].fillna(0)
ofertas['cum_prevbuys'] = ofertas['cum_prevbuys'].fillna(0)


In [13]:
ofertas[(ofertas['cum_prevbuys']==0)&(ofertas['buy']==1)]['cum_prevvisits'].describe()

count   6996.000
mean       1.064
std        1.698
min        0.000
25%        0.000
50%        0.000
75%        1.000
max       17.000
Name: cum_prevvisits, dtype: float64

In [14]:
ofertas['cum_prevvisits'].describe()

count   116534.000
mean         1.746
std          2.776
min          0.000
25%          0.000
50%          1.000
75%          2.000
max         64.000
Name: cum_prevvisits, dtype: float64

In [24]:
ofertas['cum_prevvisits'].describe()

count   119611.000
mean         1.747
std          2.753
min          0.000
25%          0.000
50%          1.000
75%          2.000
max         63.000
Name: cum_prevvisits, dtype: float64

In [15]:
np.quantile(ofertas['cum_prevvisits'],0.9)

4.0

In [ ]:
ofertas[(ofertas['cum_prevbuys']==0)&(ofertas['buy']==1)]['cum_prevvisits'].describe()

In [23]:
ofertas[(ofertas['cum_prevbuys']==0)&(ofertas['buy']==0)&(ofertas['cum_prevvisits']>17)].sort_values('cum_prevvisits',ascending=False)['name'].value_counts()

Jason Mankevich    223
Name: name, dtype: int64

In [9]:
consolidado.sort_values('acc_route_dist',ascending=False)

,id,date,offer_date,account_id,practice_name,name,visit_lon,visit_lat,acc_lon,acc_lat,state,city,status,total_offer,acc_coords,visit_coords,coords,acc_route_dist
44513,1443059,2022-01-04,2022-01-04 19:19:50+00:00,50129,Chang Vicky DDS,Eran Even-Kesef,-122.080,37.631,-75.253,39.980,PA,PHILADELPHIA,NoOffer,0.000,"(39.979604, -75.253272)","(37.63133696754376, -122.07984950205616)","[(37.701433679443724, -121.84895576110036), (3...",13003384.196
43101,1445428,2022-01-12,2022-01-12 17:43:47+00:00,49039,Reish Bruce D DDS,Eran Even-Kesef,-122.466,37.689,-75.255,41.005,PA,Stroudsburg,NoOffer,0.000,"(41.00467, -75.25503)","(37.68934705298179, -122.46620767761252)","[(37.797389758352516, -122.41015536541153), (3...",12931954.666
43342,1445047,2022-01-11,2022-01-11 19:35:23+00:00,535601,JUNKIN DENTAL ASSOC PC,Elizabeth Reynoso,-121.498,38.522,-75.114,40.137,PA,Abington,NoOffer,0.000,"(40.13683, -75.11359)","(38.5223388671875, -121.4975771774738)","[(38.594557782137024, -121.38293729411866), (3...",12895048.904
42937,1445693,2022-01-12,2022-01-12 22:33:35+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.754846555216005, -122.47661557059965)","[(37.797389758352516, -122.41015536541153), (3...",12463329.877
42757,1445964,2022-01-13,2022-01-13 18:34:05+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.75490874893754, -122.47651037771485)","[(37.524220049777156, -122.25269875328723), (3...",12462271.071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24800,1472410,2022-04-12,2022-04-12 18:45:02+00:00,445963,Gentle Dentist,Debra Morris,-86.394,39.854,-86.394,39.854,IN,Brownsburg,NoOffer,0.000,"(39.854313, -86.394336)","(39.85430908203125, -86.39433875616936)","[(39.85424242970571, -86.38305178613614), (39....",1.624
9293,1493090,2022-06-20,2022-06-20 18:38:00+00:00,444121,Gentle Dentistry: Pafford Greg DDS,Michael Beller,-112.071,33.485,-112.071,33.485,AZ,Phoenix,NoOffer,0.000,"(33.485111, -112.07118)","(33.485107421875, -112.07117545089471)","[(33.36854113017288, -111.96811773567893), (33...",1.401
33471,1459892,2022-03-03,2022-03-03 20:29:04+00:00,443972,Karren Dental Moon Valley,Michael Beller,-112.064,33.641,-112.064,33.641,AZ,Phoenix,NoOffer,0.000,"(33.640747, -112.064331)","(33.640716552734375, -112.06433769830548)","[(33.29129282880217, -112.08792881343115), (33...",1.259
10763,1491154,2022-06-14,2022-06-14 14:48:54+00:00,554398,Chawla Orthodontics,Joseph Bonham,-87.975,41.809,-87.975,41.809,IL,Westmont,NoOffer,0.000,"(41.80899, -87.975206)","(41.80902099609375, -87.97539252796021)","[(41.512115478515625, -87.94551808395231), (41...",0.853


In [7]:
consolidado.head(10)

,id,date,offer_date,account_id,practice_name,name,visit_lon,visit_lat,acc_lon,acc_lat,state,city,status,total_offer,acc_coords,visit_coords,coords,acc_route_dist
0,1505449,2022-08-02,2022-08-02 16:05:08+00:00,555801,Oral & Maxillofacial Surgical Consultants,Robert Gonshorowski,-93.501,44.974,-93.501,44.974,MN,Wayzata,NoOffer,0.000,"(44.974393, -93.50114)","(44.9737548828125, -93.50133981317549)","[(44.93621631529642, -93.4563267231858), (44.9...",81.223
1,1505448,2022-08-02,2022-08-02 16:03:12+00:00,483516,Rana Loutfy DMD STE 2,Paul Kanengiser,-74.246,40.775,-74.245,40.775,NJ,West Orange,NoOffer,0.000,"(40.77525, -74.2453)","(40.775331161022905, -74.24617652788223)","[(40.81138809681014, -74.24547999172867), (40....",3532.762
2,1505447,2022-08-02,2022-08-02 16:02:46+00:00,534785,Family Dental At Alafaya Crossings,Patrick Markowski,-81.208,28.641,-81.209,28.642,FL,Oviedo,NoOffer,0.000,"(28.641607, -81.208623)","(28.6414794921875, -81.20814392082316)","[(28.669158935546875, -81.44977291292712), (28...",160.286
3,1505445,2022-08-02,2022-08-02 15:59:57+00:00,129303,Smudde Family Dentistry,Debra Morris,-87.391,39.502,-87.391,39.502,IN,Terre Haute,NoOffer,0.000,"(39.50198, -87.39056)","(39.50238037109375, -87.39059334186669)","[(39.524824331650684, -87.12014861653236), (39...",146.362
4,1505443,2022-08-02,2022-08-02 15:59:18+00:00,459695,"Brian T. Davis, DDS",Chris Charbonneau,-84.479,34.042,-84.480,34.042,GA,Marietta,NoOffer,0.000,"(34.04169, -84.48009)","(34.04165474601674, -84.4791234433427)","[(33.95591361666666, -84.34011111666665), (33....",292.469
5,1505442,2022-08-02,2022-08-02 15:57:09+00:00,542569,LifeSmiles Dental Care,Brent Hicks,-96.683,32.933,-96.686,32.936,TX,Richardson,NoOffer,0.000,"(32.93633, -96.68615)","(32.933138686449865, -96.6828475404743)","[(33.037292569192196, -96.70129917808804), (33...",1123.341
6,1505441,2022-08-02,2022-08-02 15:56:55+00:00,541482,A Plus Dentistry,Shay Bar Nissim,-71.120,42.333,-71.120,42.333,MA,Brookline,NoOffer,0.000,"(42.33332, -71.12017)","(42.333160400390625, -71.11962313606703)","[(42.338897705078125, -71.13189251098734), (42...",158.558
7,1505439,2022-08-02,2022-08-02 15:55:36+00:00,483657,Richard Taddeo DDS,Paul Kanengiser,-74.252,40.777,-74.248,40.776,NJ,West Orange,NoOffer,0.000,"(40.77627, -74.24788)","(40.77675264007743, -74.25195171434873)","[(40.81138809681014, -74.24547999172867), (40....",2786.667
8,1505438,2022-08-02,2022-08-02 15:55:18+00:00,459980,Family Dental Care of Marietta,Chris Charbonneau,-84.474,34.041,-84.477,34.042,GA,Marietta,NoOffer,0.000,"(34.04161, -84.4765)","(34.041468065231065, -84.47360949882122)","[(33.95591361666666, -84.34011111666665), (33....",793.223
9,1505436,2022-08-02,2022-08-02 15:50:39+00:00,530349,Dental Care of Boca Raton,Vicky Barnett,-80.204,26.369,-80.197,26.368,FL,Boca Raton,Rejected,100.000,"(26.36785, -80.19716)","(26.3692799583323, -80.20358311019857)","[(26.39424570838585, -80.15240035958603), (26....",515.793


In [2]:
(BigQuery_client.query("""
SELECT *
FROM `star-big-data.star_us_rds.ofertas` o
LIMIT 100
""").result().to_dataframe(create_bqstorage_client=True,))

,wc_pd_grams,forex_rate,yd_au_grams,yd_pt_percents,yd_pd_grams,territory_id,wc_au_grams,wd_ag_grams,wd_weight_loss,spendlog_audit_id,yr_ag_percents,high_nickel_input_values_array_id,yr_pd_grams,extendedstatus,wd_au_percents,antimoneylaunderingdata_id,yc_pd_grams,yr_ag_grams,discount,wc_pd_percents,yd_ag_percents,yc_au_grams,pt_per_oz,wc_pt_percents,ag_dollars,yd_au_percents,longitude,pt_dollars,_sdc_deleted_at,refineinfoforwhite_id,yr_pt_grams,account_data,outcome_details,id,pt_per_g,offer_currency,yd_pt_grams,latitude,yr_grams,wc_ag_percents,wc_weight_loss,yd_pd_percents,au_adjustment_dollars,yd_weight_loss,au_dollars,ag_adjustment_dollars,_sdc_table_version,uuid,yr_weight_loss,melt_id,yc_pd_percents,au_per_g,ag_per_g,yd_ag_grams,forex_adjustment,updated_at,status,total_offer_in_offer_currency,rep_id,wd_pt_grams,refineinfoforyellow_id,yc_grams,yr_au_grams,wd_grams,_sdc_received_at,visit_id,_sdc_sequence,pd_adjustment_dollars,yr_pt_percents,au_per_oz,signature_url,total_offer,wd_au_grams,photo_urls,wd_pd_grams,wc_au_percents,created_at,wc_ag_grams,wc_pt_grams,pd_per_oz,yc_weight_loss,ag_per_oz,yc_ag_percents,wd_pd_percents,pd_per_g,yc_pt_percents,yr_pd_percents,yc_ag_grams,yd_grams,account_id,_sdc_batched_at,offer_date,_sdc_extracted_at,input_values_array_id,yc_au_percents,wd_pt_percents,yc_pt_grams,yr_au_percents,pd_dollars,wc_grams,pt_adjustment_dollars,wd_ag_percents,commission_exception
0,0.000,None,0.000,0.015,0.000,641,0.000,0.960000000,0E-9,NaN,0.110000000,NaN,0E-9,None,0.110000000,NaN,0.140,0E-9,NaN,0.191,0.130,1.960,889.000,0.004,0.336,0.460,-87.099,0.570,None,42346.000,0E-9,None,None,1505365,8.140,None,0.000,39.531,0.000,0.064,0.000,0.028,11.260,0.000,63.151,0.060,1633421232123,22798A14-D0DA-4B1E-A40D-416465F90A7D,0E-9,NaN,0.035,17.640,0.200,0.000,None,2022-08-02 14:26:09+00:00,Accepted,NaN,300,0.060000000,42345.000,4.000,0E-9,15.000,2022-08-02 14:35:49.562000+00:00,22888.000,1659450931641267014,11.280,0.010000000,1753.400,https://s3.amazonaws.com/ic3-test/offerPhotos/...,150.000,1.620000000,[],2.860000000,0.108,2022-08-02 14:26:08+00:00,0.000,0.000,2093.000,0.000,20.065,0.180,0.190000000,21.080,0.002,0.020000000,0.720,0.000,446313,2022-08-02 14:41:34.506000+00:00,2022-08-02 14:26:08+00:00,2022-08-02 14:35:30.671000+00:00,NaN,0.490,0E-9,0.010,0.350000000,63.240,0.000,0.100,0.060000000,False
1,1.610,None,0.000,0.015,0.000,634,1.050,2.930000000,0E-9,NaN,0.110000000,NaN,0E-9,None,0.100000000,NaN,0.000,0E-9,NaN,0.161,0.130,0.000,908.000,0.003,0.644,0.460,-81.981,1.460,None,42339.000,0E-9,None,None,1505361,8.110,None,0.000,34.939,0.000,0.065,0.000,0.028,0.660,0.000,88.339,0.000,1633421232123,8C7F421A-376F-4BB6-8269-A9E155FDA82D,0E-9,NaN,0.035,15.310,0.180,0.000,None,2022-08-02 14:15:44+00:00,Accepted,NaN,69,0.150000000,NaN,0.000,0E-9,45.000,2022-08-02 14:35:49.562000+00:00,22887.000,1659450931641196009,1.240,0.010000000,1772.400,https://s3.amazonaws.com/ic3-test/offerPhotos/...,260.000,4.720000000,[],7.260000000,0.105,2022-08-02 14:15:42+00:00,0.650,0.030,2182.000,0.000,20.405,0.180,0.160000000,18.900,0.002,0.020000000,0.000,0.000,543561,2022-08-02 14:41:34.483000+00:00,2022-08-02 14:15:42+00:00,2022-08-02 14:35:30.671000+00:00,NaN,0.490,0E-9,0.000,0.350000000,167.643,10.000,0.010,0.070000000,False
2,NaN,None,NaN,NaN,NaN,611,NaN,None,None,NaN,None,NaN,None,Does Not Collect,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-84.411,NaN,None,NaN,None,None,"{""displayName"":""Other"",""reason"":""Other""}",1505363,NaN,None,NaN,33.949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633421232123,F6098DFA-B8CD-473B-B5BC-DAE22255CFED,None,NaN,NaN,NaN,NaN,NaN,None,NaT,NoOffer,NaN,289,None,NaN,NaN,None,NaN,2022-08-02 14:35:49.562000+00:00,NaN,1659450931641225011,NaN,None,NaN,None,0.000,None,[],None,NaN,2022-08-02 14:23:27+00:00,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,NaN,556892,2022-08-02 14:41:34.486000+00:00,2022-08-02 14:23:28+00:00,2022-08-02 14:35:30.671000+00:00,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,False
3,NaN,None,NaN,NaN,NaN,639,N

In [46]:
consolidado['acc_route_dist'].describe().round()

count      44710.000
mean        5530.000
std       175878.000
min            0.000
25%          140.000
50%          354.000
75%          724.000
max     13003384.000
Name: acc_route_dist, dtype: float64

In [54]:
np.quantile(consolidado['acc_route_dist'],0.87)

1065.4488717477482

In [48]:
consolidado[(consolidado['acc_route_dist']>10000)&(consolidado['acc_route_dist']<13003385)].sort_values('acc_route_dist',ascending=False).head()

,id,date,offer_date,account_id,practice_name,name,visit_lon,visit_lat,acc_lon,acc_lat,state,city,status,total_offer,acc_coords,visit_coords,coords,acc_route_dist
44397,1443059,2022-01-04,2022-01-04 19:19:50+00:00,50129,Chang Vicky DDS,Eran Even-Kesef,-122.080,37.631,-75.253,39.980,PA,PHILADELPHIA,NoOffer,0.000,"(39.979604, -75.253272)","(37.63133696754376, -122.07984950205616)","[(37.701433679443724, -121.84895576110036), (3...",13003384.196
42985,1445428,2022-01-12,2022-01-12 17:43:47+00:00,49039,Reish Bruce D DDS,Eran Even-Kesef,-122.466,37.689,-75.255,41.005,PA,Stroudsburg,NoOffer,0.000,"(41.00467, -75.25503)","(37.68934705298179, -122.46620767761252)","[(37.797389758352516, -122.41015536541153), (3...",12931954.666
43226,1445047,2022-01-11,2022-01-11 19:35:23+00:00,535601,JUNKIN DENTAL ASSOC PC,Elizabeth Reynoso,-121.498,38.522,-75.114,40.137,PA,Abington,NoOffer,0.000,"(40.13683, -75.11359)","(38.5223388671875, -121.4975771774738)","[(38.594557782137024, -121.38293729411866), (3...",12895048.904
42821,1445693,2022-01-12,2022-01-12 22:33:35+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.754846555216005, -122.47661557059965)","[(37.797389758352516, -122.41015536541153), (3...",12463329.877
42641,1445964,2022-01-13,2022-01-13 18:34:05+00:00,49623,Associated Dental Arts of Oswego,Eran Even-Kesef,-122.477,37.755,-76.533,43.451,NY,Oswego,NoOffer,0.000,"(43.45149, -76.53278)","(37.75490874893754, -122.47651037771485)","[(37.524220049777156, -122.25269875328723), (3...",12462271.071


In [49]:
consolidado.loc[(consolidado['acc_route_dist']>10000)&(consolidado['acc_route_dist']<13003385),'name'].value_counts()

Jim Harness            110
Brent Hicks             95
Sammie Harris           88
Patrick Markowski       35
Jason Mankevich         33
Fernando Andrews        32
Judd Stewart            29
Brian Sundberg          29
Whitney Ross            27
Shay Bar Nissim         25
Vicky Barnett           25
Eran Even-Kesef         24
Elizabeth Reynoso       24
Robert Gonshorowski     23
Alex Aguilar            21
Debra Morris            20
Demery Moody            19
Robert Gregg            18
Eric Moskow             18
Michael Beller          18
Jordan Hunter           18
Joseph Bonham           16
Chris Charbonneau       13
Aaron Domke             12
Troy C-Rep              10
David Gulakowski         7
Paul Kanengiser          3
Loren Feldman            2
Name: name, dtype: int64

In [119]:
index = 43224
latlon = list(consolidado['coords'][index])
mapit = folium.Map( location=latlon[0], zoom_start=6 ,tiles='Stamen Toner')
for coord in latlon:
    folium.Circle( location=[ coord[0], coord[1] ], color='#43d9de', radius=10 ).add_to( mapit )
folium.Circle(location=[consolidado['visit_coords'][index][0],consolidado['visit_coords'][index][1]],color='red',radius=10).add_to( mapit )
folium.Circle(location=[consolidado['acc_coords'][index][0],consolidado['acc_coords'][index][1]],color='yellow',radius=10).add_to( mapit )
mapit.fit_bounds(mapit.get_bounds())
mapit